In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.preprocessing import image
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, smart_resize
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import cv2
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from keras.metrics import AUC
from tqdm.auto import tqdm
sns.set_style('darkgrid')
pd.set_option("display.max_columns", None)

In [ ]:
train_dir= '../input/resized-plant2021/img_sz_384'
test_dir =  '../input/plant-pathology-2021-fgvc8/test_images'
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
train['labels'] = train['labels'].apply(lambda string: string.split(' '))

In [ ]:
s = list(train['labels'])
mlb = MultiLabelBinarizer()
trainx = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=train.index)
print(trainx.columns)

In [ ]:
labels = pd.concat([train['image'], trainx], axis=1)
labels.head()

In [ ]:
image_datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=5,
    zoom_range=0.1,
    shear_range=0.05,
    horizontal_flip=True,
    validation_split=0.1
    
)
IMAGE = (256, 256)
BATCH_SIZE = 64

In [ ]:
train_data = image_datagen.flow_from_dataframe(
    dataframe=labels,
    directory= '../input/resized-plant2021/img_sz_512',
    x_col="image",
    y_col=labels.columns.tolist()[1:],
    color_mode="rgb",
    target_size = IMAGE,
    class_mode="raw",
    subset = "training",
    batch_size=BATCH_SIZE
)

test_data = image_datagen.flow_from_dataframe(
    dataframe=labels,
    directory= '../input/resized-plant2021/img_sz_512',
    x_col="image",
    y_col=labels.columns.tolist()[1:],
    color_mode="rgb",
    target_size = IMAGE,
    class_mode="raw",
    subset = "validation",
    batch_size=BATCH_SIZE
)

# Modeling

In [ ]:
inputs = tf.keras.Input(shape=(256, 256, 3))
x = tf.keras.applications.MobileNetV2(include_top=False)(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(6, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs, outputs)
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(lr=1e-4),
              metrics=["accuracy"])

In [ ]:
rlp = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                           mode='min',
                                           patience=2, 
                                           verbose=0, 
                                           factor=0.01)
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                             patience=5, 
                                             verbose=1, 
                                             restore_best_weights=True)
callbacks=[rlp,earlystop]

In [ ]:
model_history = model.fit(train_data, 
                          validation_data=test_data, 
                          validation_steps = test_data.n // BATCH_SIZE,
                          epochs=30, 
                          callbacks=callbacks)

In [ ]:
fix, ax = plt.subplots(figsize=(20, 6))
pd.DataFrame(model_history.history)[['loss', 'val_loss']].plot(ax=ax, title='Model Loss Curve')

In [ ]:
model.save('mobilenetv2image512.h5')

In [ ]:
from sklearn.metrics import classification_report

# Submission

In [ ]:
submissions = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submissions.head()

In [ ]:
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_data_generator.flow_from_dataframe(
    submissions,
    directory = '../input/plant-pathology-2021-fgvc8/test_images',
    x_col="image",
    y_col=None,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode=None,
    shuffle=False,
    batch_size=32
)

predictions = model.predict(test_generator)

In [ ]:
verdict = (predictions>0.25)

In [ ]:
for x in verdict:
    count = 0
    for i in range(len(x)):
        if x[i]==False:
            count=count+1
    if count==len(x):
        x[2]=True

In [ ]:
for x in verdict:
    if x[2]:
        for i in range(len(x)):
            x[i]=False
        x[2]=True

In [ ]:
label = labels.columns.tolist()[1:]
label

In [ ]:
pred_lists = []
for i in range(verdict.shape[0]):
    tmp = []
    for j, c in enumerate(label):
        if verdict[i, j]:
            tmp.append(c)
    pred_lists.append(tmp)

pred_lists = [' '.join(t) for t in pred_lists]
pred_lists

In [ ]:
submissions['labels'] = np.array(pred_lists)

In [ ]:
submissions

In [ ]:
submissions.to_csv('submission.csv', index=False)  